<a href="https://colab.research.google.com/github/masayasato0407/pycox_DeepSurv/blob/main/pycox_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#pycoxを用いたDeepSurv予測

モデル訓練用のtrain.csv、ハイパーパラメーター (今回はlearning rate[学習率])を設定するためのvalidation.csv、精度評価用のtest.csvを使用します。

In [ ]:
#pycoxのインストール
!pip install pycox

In [ ]:
#必要なライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

import torch
import torchtuples as tt

from pycox.datasets import metabric
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [ ]:
# ローカルファイルを直接アップロードする (train data)
from google.colab import files
uploaded = files.upload()

In [ ]:
#training dataの読み込み (df_trainという名前で読み込む)
df_train=pd.read_csv ("train.csv",encoding="utf-8")
df_train.head

In [ ]:
# ローカルファイルを直接アップロードする (validation data)
from google.colab import files
uploaded = files.upload()

In [ ]:
#validation dataの読み込み (df_valという名前で読み込む)
df_val=pd.read_csv ("validation.csv",encoding="utf-8")
df_val.head

In [ ]:
# ローカルファイルを直接アップロードする (test data)
from google.colab import files
uploaded = files.upload()

In [ ]:
#test dataの読み込み (df_testという名前で読み込む)
df_test=pd.read_csv ("test.csv",encoding="utf-8")
df_test.head

In [ ]:
#今回は正規化をしないので、以下のコードを書く
cols_leave = ['dis1', 'con1','con2', 'dis2', 'con3', 'con4','con5', 'con6', 'con7', 'con8','con9', 'dis3']
leave = [(col, None) for col in cols_leave]
x_mapper = DataFrameMapper(leave)

In [ ]:
#ライブラリの制約上、float (実数を扱うことができるデータ型)32に変換する
x_train = x_mapper.fit_transform(df_train).astype('float32')
x_val = x_mapper.transform(df_val).astype('float32')
x_test = x_mapper.transform(df_test).astype('float32')

In [ ]:
#target (timeとevent)を設定する
get_target = lambda df: (df['time'].values, df['event'].values)
y_train = get_target(df_train)
y_val = get_target(df_val)
durations_test, events_test = get_target(df_test)
val = x_val, y_val

In [ ]:
#モデルの作成
in_features = x_train.shape[1]
num_nodes = [32, 32]
out_features = 1
batch_norm = True
dropout = 0.1
output_bias = False

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                              dropout, output_bias=output_bias)

In [ ]:
#モデルのトレーニング 
#最適化のためのソフトウェア (オプティマイザーの指定が必要)→以下のtt.optim.Adam
model = CoxPH(net, tt.optim.Adam)

In [ ]:
#最適なlearning rateを見つける
batch_size = 256
lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance=10)
_ = lrfinder.plot()

In [ ]:
#最適なlearning rateの提示
lrfinder.get_best_lr()

In [ ]:
#最適なlearning rateの設定
#learning rateの最適化は少し高めになることもあり、こちらを採用せず手動で0.01にすることも多い
model.optimizer.set_lr(0.042)

In [ ]:
#epoch数などの設定
#検証損失が改善しなくなった時点でトレーニングを停止するEarlyStoppingを設定 (2行目)
#verbose→学習のログの表示の有無
epochs = 512
callbacks = [tt.callbacks.EarlyStopping()]
verbose = True

In [ ]:
#学習を行う
%%time
log = model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose,
                val_data=val, val_batch_size=batch_size)

In [ ]:
#評価のために、まずテストセットの生存推定値を得る必要があります。
_ = model.compute_baseline_hazards()

In [ ]:
#生存推定値をデータフレームとして返す model.predict_surv_df で行う
surv = model.predict_surv_df(x_test)

In [ ]:
#Evaluation
#We can use the EvalSurv class for evaluation the concordance, brier score and binomial log-likelihood. Setting censor_surv='km' means that we estimate the censoring distribution by Kaplan-Meier on the test set.
ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')

In [ ]:
#精度評価
#EvalSurvクラスは，concordance，brier score，二項対数尤度の評価に用いることができます
#censor_surv='km'とすると、テストセットに対してKaplan-Meierによる打ち切り分布の推定を行うことになります。
ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')

In [ ]:
#testdataにおけるcindexの表示
ev.concordance_td()

In [ ]:
#testデータセットの説明変数を用いてtest setのサンプル0-4 (1行目のコードで指定)の予測の表示
surv.iloc[:, :5].plot()
plt.ylabel('S(t | x)')
_ = plt.xlabel('Time')